In [6]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import cv2
import numpy as np
from PIL import Image

In [21]:
# Load the pre-trained Inception-ResNet-V2 model
base_model = tf.keras.applications.InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))

# Freeze the pre-trained weights
base_model.trainable = False

# Define the input shape
input_shape = (224, 224, 3)

# Define the input layer
inputs = tf.keras.layers.Input(shape=input_shape)

# Pass the input layer through the pre-trained model
features = base_model(inputs)

# Add a dense layer for multi-class classification
# outputs = tf.keras.layers.Dense(units=8, activation='sigmoid')(features)

outputs = tf.keras.layers.Flatten()(features)
outputs = tf.keras.layers.Dense(units=8, activation='sigmoid')(outputs)

In [22]:
# Create the model
model = tf.keras.Model(inputs=inputs, outputs=outputs)
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
model.summary()


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 inception_resnet_v2 (Functi  (None, 5, 5, 1536)       54336736  
 onal)                                                           
                                                                 
 flatten_1 (Flatten)         (None, 38400)             0         
                                                                 
 dense_2 (Dense)             (None, 8)                 307208    
                                                                 
Total params: 54,643,944
Trainable params: 307,208
Non-trainable params: 54,336,736
_________________________________________________________________


In [9]:
# Load the image data and one-hot encoded multi-label data
# Load the data from the excel sheet into a pandas dataframe
# Load both left and right eye dataset
base_path = 'C:/Users/manik/DL-Project/DeepLearning/DL-Project'
left_df = pd.read_excel(f'{base_path}/updated_left_eye.xlsx')
right_df = pd.read_excel(f'{base_path}/updated_right_eye.xlsx')

#concating both dataframes
df = pd.concat([left_df, right_df], axis=0)


# Split the data into two arrays, one for the image paths and one for the labels
image_paths = df["Fundus"].values
labels = df.iloc[:, 3:].values

# Convert the lists to tensors
image_paths = tf.constant(image_paths)
labels = tf.constant(labels)

# Create a dataset from the image paths and labels
dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))

model.summary()

In [10]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 inception_resnet_v2 (Functi  (None, 5, 5, 1536)       54336736  
 onal)                                                           
                                                                 
 dense_1 (Dense)             (None, 5, 5, 8)           12296     
                                                                 
Total params: 54,349,032
Trainable params: 12,296
Non-trainable params: 54,336,736
_________________________________________________________________


In [24]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 inception_resnet_v2 (Functi  (None, 5, 5, 1536)       54336736  
 onal)                                                           
                                                                 
 flatten_1 (Flatten)         (None, 38400)             0         
                                                                 
 dense_2 (Dense)             (None, 8)                 307208    
                                                                 
Total params: 54,643,944
Trainable params: 307,208
Non-trainable params: 54,336,736
_________________________________________________________________


In [11]:
# Load the images and preprocess them
def load_and_preprocess_image(file_path, label):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.keras.applications.inception_resnet_v2.preprocess_input(image)
    return image, label

# Apply the load_and_preprocess_image function to the dataset
dataset = dataset.map(load_and_preprocess_image)

# Shuffle the data and batch it
batch_size = 32
dataset = dataset.shuffle(buffer_size=len(image_paths)).batch(batch_size)

In [12]:
dataset

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 8), dtype=tf.int64, name=None))>

In [ ]:
# Split the data into training and validation sets
# x_train, x_val, y_train, y_val = train_test_split(images, labels, test_size=0.2)

In [17]:
# Train the model
history=model.fit(dataset)

ValueError: in user code:

    File "C:\Users\manik\anaconda3\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\manik\anaconda3\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\manik\anaconda3\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\manik\anaconda3\lib\site-packages\keras\engine\training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\manik\anaconda3\lib\site-packages\keras\engine\training.py", line 1082, in compute_loss
        return self.compiled_loss(
    File "C:\Users\manik\anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "